# Basics of NLP: DataFrame rudiments and spaCy

## Libraries 

We will consider a corpus of radio recordings in french (source: France inter, RFI, ...) :  RADIOS.txt


The first step in writing python code is determining which libraries we want to use. To stay organized, we generally import all libraries at the top (of a notebook or a standalone python file).

In [ ]:
#import four important libraries
import pandas as pd
#pandas is the most popular data science library. It is used
#for organizing data into columns.
#See more here: https://pandas.pydata.org/

import re
#re is the python library for regular expressions. It is
#difficult to master, but very useful.
#See more here: https://docs.python.org/3/library/re.html

import matplotlib.pyplot as plt
#matplotlib is the most popular and simple library for
#scientific plots in python. The pyplot submodule is
#a data scientist's bread and butter for data visualization.
#See more here: https://matplotlib.org/2.0.2/api/pyplot_api.html

import xml.etree.ElementTree as ET
#ElementTree is a library for parsing XML in python.
#This is a useful skill for data scientists, as lots of data
#is published in XML form. There are other libraries for this,
#such as untangle or xmltodict; however, ElementTree is lower level.
#See more here: https://docs.python.org/3/library/xml.etree.elementtree.html

from numpy import log
#we will need a logarithm function; numpy is a standard mathematical
#library

## Pandas Dataframe basics

In this TP we use Pandas [DataFrames](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) to manage our dataset. Pandas is a standard tool in data science.

In [ ]:
#read the dataset as a DataFrame
df_radios = pd.read_csv('RADIOS.TXT', names=['text'])

In [ ]:
print(len(df_radios))
df_radios.head()

We can list the columns of a dataset ````df```` by calling ````df.columns````.

**Question**: What are the columns of the dataset ````df_radios````?

In [ ]:
df_radios.columns
#todo: explain each column

We can access to a specific using ````df.loc[i]```` or ````df.iloc[i]```` 


**Question**: What is the difference between .loc and .iloc?

In [ ]:
#todo: save the text value of the 5th element of our dataset into the one_text variable. 
one_text = '' #todo 

We can count the number of words in a string using the ````len```` function and ````split```` method of strings. For example:


```
string = "bla bla bla"
len(string.split(" "))
>>> 3
```

**Question**: Fill in the following function and run it for a our ````one_text```` example. 






In [ ]:
def num_words(sentence):
    '''
    Calculate the number of words in the sentence.
    '''
    #todo

num_words(one_text)


Furthermore, we can calculate values for each row, based on a specific column, using the ````apply```` method:

```
lengths = df.column_name.apply(lambda text: num_words(text))
```

This will give us a [pandas Series](https://pandas.pydata.org/docs/reference/api/pandas.Series.html) of the lengths of each text.

**Question**: Add the length of each text as a new column in the dataframe (see [pandas docs](https://pandas.pydata.org/docs/user_guide/dsintro.html#basics-dataframe-sel-add-del) for details). 

**Question**: What is the distribution of the length of the texts in words? Visualize this using a python plot (see [plt.hist](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html)).



In [ ]:
# todo

lengths = '' #todo 

## to plot 
plt.hist(lengths,bins=100) 

We can filter a dataframe using criteria based on values in certain columns. The basic syntax for this is:

````
new_df = df[df["column"].apply(lambda row_value: my_func(row_value) == some_value)]
````

This gives a new DataFrame composed of all elements of ````df```` such the values of the function ````myfunc````, calculated for the value of column "column" for each row is equal to some_value. For example, if we wanted all rows that begin with "a", we could run:

````
df_letter_A = df[df["text"].apply(lambda text: text[0] == "a")]
````

**Question**: Retain only documents with at least 50 words

In [ ]:
#todo


### Finding useful information in the text:

We will implement regular expresions (regex) to filter the text according to a specific condiction. 

To implement the regex evaluation in Pandas we use the [`Series.str.contains`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.contains.html) filtering function, as :

```` df.column_name.str.contains('[a-z]', regex=True)````

Also useful: `match`, `Series.str.startswith`, `Series.str.endswith`


**Question**  : How many texts are there with at least one word of 9 letters in RADIOS.txt?

**Question**  : Are there any texts with words without vowels in RADIOS.txt?


You can use the following regexs : 

| regex | Description |
|--------|--------------------|
|`'[A-Z]'`|filtre lignes contenant une majuscule|
|`'^[A-Z]'`|filtre lignes commençant par une majuscule|
|`'[A-Z]$'`|filtre lignes finissant par une majuscule|
|`'^[A-Z]*$'`|filtre lignes entièrement majuscules|
|`'[aeiouAEIOU]'`|filtre lignes contenant une voyelle|
|`'^[aeiouAEIOU]'`|filtre lignes commençant par une voyelle|
|`'[aeiouAEIOU]$'`|filtre lignes finissant par une voyelle|
|`'^[^aeiouAEIOU]'`|filtre lignes commençant par une non-voyelle|
|`'[^aeiouAEIOU]$'`|filtre lignes finissant par une non-voyelle|
|`'[aeiouAEIOU].*[aeiouAEIOU]'`|filtre lignes avec au moins deux voyelles|
|`'^[^aeiouAEIOU]*[aeiouAEIOU][^aeiouAEIOU]*$'`| filtre lignes avec exactement une voyelle|

Avec expressions régulières:

| Expression | Match |
|--------|--------------------|
|`a`|la lettre "a"|
|`[a-z]`|une lettre minuscule|
|`[A-Z]`|une lettre majuscule|
|`[0-9]`|un chiffre|
|`[0123456789]`|un chiffre|
|`[aeiouAEIOU]`|une voyelle|
|`[^aeiouAEIOU]`|tout sauf une voyelle|
|`.`|un caractère|
|`^`|début de ligne|
|`$`|fin de ligne|
|`x*`| "x" répété 0 fois ou plus|
|``x+``| "x" répété 1 fois ou plus|
|`x\|y`| "x" ou "y"|

In [ ]:
## For example: 
df_radios['a-z'] = df_radios.text.str.contains('[a-z]', regex=True)
df_radios[df_radios['a-z']]

## Vocabulary 

We will create a a dictionary as the vocabulary for our corpus of texts. 
Create a dictionary where each key is a word and the value will be its frequency. 

```` texts_vocab = {'key1':value, 'key2':value} ````

**Question** Create a Vocabulary of bigrams and trigrams 

**Question** How many different words exist in our corpus of texts?

**Question**: What is the frequency distribution of the words? use ```plt.hist```

**Question** [optional] How many bigrams and trigrams exist? Visualize the frequency distribution. 



In [ ]:
# todo 

## Introduction to spaCy

spaCy is a package that performs some basic NLP tasks for us, based on training on a large corpus. We can import spaCy libraries for different languages and of different complexities (often in small, medium, and large sizes, where larger means slower to use but with slightly more precise behaviour). For our use case, we will "fr_core_news_sm", which is the small French language  model.

For basic questions about spaCy, refer to the documentation [here](https://spacy.io/usage/spacy-101).

In [ ]:
!python -m spacy download fr_core_news_sm

In [ ]:
import spacy
from spacy.lang.fr.examples import sentences 
# nlp = spacy.load("en_core_web_sm") 
nlp = spacy.load("fr_core_news_sm")  #this could take a minute to download the first time


The ````nlp```` object can be used to analyze documents. For example:

````doc_analyzed = nlp("Hello world")
doc_analyzed
>>> Hello world
type(doc_analyzed)
>>> spacy.tokens.doc.Doc
````

In [ ]:
doc = nlp(one_text)
type(doc)


**Question**: Use spaCy to list each token in the ````text```` of the document with id 1500.

*Hint: spacy.tokens.doc.Doc is an iterable*

In [ ]:
#todo 


**Question**: List the POS of every token from the previous exercise. Do you agree with spaCy?

*Hint: use token.pos_*

In [ ]:
#todo

The lemma of a term is the simplest and most distilled version of the term, without prefixes, suffixes, etc. Converting terms to lemmas can be useful in order to reduce the vocabulary size of a corpus and group terms into equivalence classes. For example, "run", "runs", "running", and "ran" all share roughly the same semantic meaning, and in some contexts (though not all!) should be treated the same way.

**Question**: extract the lemmas from the previous exercise. Which terms stay the same, and which differ? In what ways do they differ?

*Hint: use token.lemma_*

In [ ]:
#todo

**Question**: How many unique terms are there among all the texts? How many unique *lemmas* are there among all the texts?

In [ ]:
#todo

**Question**: Plot the distribution of POS over the all the texts.

In [ ]:
#todo

**Question**: Plot the distribution of the 100 most common words over the all the texts, provided that they are not space or punctuation. Use a log scale for the y axes and describe the graph: the y axis should be frequency, and the x axis should be words sorted by frequency.

*Hint: see [Zipf's law](https://en.wikipedia.org/wiki/Zipf%27s_law)*

In [ ]:
#todo

## Stop words

Stop words are words that, under some frameworks, contribute marginal semantic value, but are important for syntax. For example, "the", "and", "such", etc.

**Question**: In what kind of analysis can we strike stop words without worrying about losing information? In what kind of analysis might we want to keep them around?


**Question**: Write a function:

````def clean(sentence)````

that strikes all stop words, replaces each remaining token with its lemma. Create a cleaned version of each abstract, and save this cleaned version as a new column of the dataframe called "text_clean".

In [ ]:
def clean(sentence):
    #todo
    pass

df_radios["text_clean"] = df_radios.text.apply(clean)

## Dependencies
spaCy calculates dependencies between different tokens:

In [ ]:
# exp_doc = nlp("Hello! I hope you are well.")
exp_doc = nlp(one_text)
for word in exp_doc:
   print(word.text, word.tag_, word.head)

We can visualize this using dispacy, a visualization library:

In [ ]:
from spacy import displacy
displacy.render(exp_doc, style='dep', jupyter=True, options={'distance': 130})


**Question**: Visualize a random title from the dataframe - describe the visualization.

## NER

spaCy can determine named entities.

In [ ]:

for ent in nlp("Joe Biden is the president of the USA").ents:
    print("ENT: ", ent.text, ent.label_, spacy.explain(ent.label_))


**Question**: calculate the ten most common named entities and the five most common labels among the texts.

In [ ]:
#todo

## TF-IDF

One of the most important statistics we can calculate for a document is the TF-IDF score. This stands for "term frequency-inverse document frequency" score, and it is a measure of the importance of each word to a given document, in the context of the corpus of documents. For each word in each document, we calculate the following ratio:

$\frac{TF_{word; doc}}{IDF_{word; doc}}$

$TF_{word; doc} = \frac{\text{Total counts of word in doc}}{\text{Number of terms in doc}}$

$IDF_{word; doc} = \log{\left(\frac{|\text{corpus}|}{\text{Number of docs containing word}}\right)}$

To have a high TF-IDF score in a document, a word must occur frequently in a document, while appearing rarely in others.

**Question**: Give the limits of TF and IDF for a word/document pair, and give an example of a word-document pair where this might occur (for example, word="Hello", doc=Wikipedia page for greetings, corpus=all of Wikipedia).


**Question**: Code the functions TF and IDF:


In [ ]:
def tf(word, document):
    #todo
    pass



def idf(word, all_documents):
    #todo
    pass



def tf_idf(word, document, all_documents):
    return tf(word, document)/idf(word, all_documents)

#test your code:
my_text = df_radios.iloc[0].text_clean
tf_idf("radio", my_text, df_radios.text_clean)

**Question**: What is the word with the highest and lowest TF-IDF score?

**Question**: If we took each document and duplicated it in place, i.e.:

a document "hello world" would become "hello world hello world", how would the tf-idf score of any document-word pair change?

What about if instead of duplicating in place, we doubled the size of the corpus, i.e.

if document "hello world" is in the corpus once, we add a second copy so now it is in the corpus twice. How would the tf-idf score of any document-word pair change now?

**Question**: What could be a potential use-case for the TF-IDF score in practice?

### Final Comments 

Add your conclusions about this lab. You can include your level of satisfation with your work and your opinion on the length and difficulty of the exercises.